In [1]:
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.metrics import (
    f1_score, 
    precision_score, 
    recall_score, 
    accuracy_score, 
    classification_report
)
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pickle

In [2]:
data = pd.read_pickle("/Users/mac/Desktop/Code_Smell_Detection/dataset/data_class/codeT5_embeddings/processed_embeddings/codeT5_small_embed_line_by_line_avg.pkl")
data.head()

,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,emb_10,...,emb_506,emb_507,emb_508,emb_509,emb_510,emb_511,emb_512,label,sample_id,severity
0,0.203212,-0.158422,-0.389438,0.149843,-0.091707,-0.070891,-0.509365,0.128811,0.667703,0.125086,...,-0.357746,-0.027873,-0.060680,-0.374299,-0.006711,0.017262,0.171716,1,8077,critical
1,0.153069,0.010321,-0.269168,0.164093,0.074495,-0.212439,-0.503994,0.281666,0.639152,0.116590,...,0.197503,-0.313194,0.018110,-0.075032,-0.256461,-0.257803,0.129895,1,5553,critical
2,0.104762,0.127288,-0.383993,0.027451,-0.075439,-0.129693,-0.476065,0.157045,0.511468,0.051980,...,0.075900,-0.401862,-0.156971,-0.051620,-0.359596,-0.073411,0.087994,1,9341,critical
3,0.294902,-0.141709,-0.377877,0.008796,0.014903,-0.117621,-0.442314,0.250533,0.420496,0.272366,...,0.165051,-0.262370,0.028658,-0.063762,-0.251821,0.021107,0.076305,1,10419,critical
4,0.202409,-0.155407,-0.381511,0.158429,-0.101959,-0.095471,-0.511652,0.140237,0.668266,0.126936,...,-0.362875,-0.007050,-0.052576,-0.390970,-0.020881,0.020677,0.152538,1,12232,critical


In [3]:
# Separate features and labels
X = data[[col for col in data.columns if col.startswith("emb_")]].values  # Select embedding columns
y = data['label'].values 

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define classifiers
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
bagging = BaggingClassifier(estimator=RandomForestClassifier(), n_estimators=100, random_state=42)  
xgboost = xgb.XGBClassifier(n_estimators=100, random_state=42)

# Add the classifiers to a dictionary
classifiers = {
    "Random Forest": random_forest,
    "Bagging": bagging,
    "XGBoost": xgboost
}

In [4]:
# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Dictionary to store cross-validation results
results = {}

# Perform cross-validation for each classifier
for clf_name, clf in classifiers.items():
    metrics_summary = {
        "f1_micro": [],
        "f1_macro": [],
        "accuracy": [],
        "precision_macro": [],
        "recall_macro": []
    }
    
    for train_idx, val_idx in skf.split(X_train, y_train):
        # Split the training data into training and validation folds
        X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
        y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]
        
        # Train the classifier
        clf.fit(X_train_fold, y_train_fold)
        
        # Make predictions on the validation set
        y_pred = clf.predict(X_val_fold)
        
        # Calculate metrics
        metrics_summary["f1_micro"].append(f1_score(y_val_fold, y_pred, average='micro'))
        metrics_summary["f1_macro"].append(f1_score(y_val_fold, y_pred, average='macro'))
        metrics_summary["accuracy"].append(accuracy_score(y_val_fold, y_pred))
        metrics_summary["precision_macro"].append(precision_score(y_val_fold, y_pred, average='macro'))
        metrics_summary["recall_macro"].append(recall_score(y_val_fold, y_pred, average='macro'))
    
    # Store mean metrics for the classifier
    results[clf_name] = {metric: np.mean(scores) for metric, scores in metrics_summary.items()}

# Print cross-validation results
print("\nCross-Validation Results:")
for clf_name, metrics in results.items():
    print(f"\n{clf_name} Metrics:")
    for metric, value in metrics.items():
        print(f"{metric.capitalize()}: {value:.4f}")


Cross-Validation Results:

Random Forest Metrics:
F1_micro: 0.7985
F1_macro: 0.6764
Accuracy: 0.7985
Precision_macro: 0.7831
Recall_macro: 0.6547

Bagging Metrics:
F1_micro: 0.7977
F1_macro: 0.6619
Accuracy: 0.7977
Precision_macro: 0.8044
Recall_macro: 0.6416

XGBoost Metrics:
F1_micro: 0.8159
F1_macro: 0.7393
Accuracy: 0.8159
Precision_macro: 0.7767
Recall_macro: 0.7205


In [5]:
# Final evaluation on the test set
final_results = {}

for clf_name, clf in classifiers.items():
    # Train the classifier on the entire training set
    clf.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_test = clf.predict(X_test)
    
    # Calculate test metrics
    f1_micro_test = f1_score(y_test, y_pred_test, average='micro')
    f1_macro_test = f1_score(y_test, y_pred_test, average='macro')
    precision_test = precision_score(y_test, y_pred_test, average='macro')
    recall_test = recall_score(y_test, y_pred_test, average='macro')
    accuracy_test = accuracy_score(y_test, y_pred_test)
    report_test = classification_report(y_test, y_pred_test)
    
    # Save final metrics for the classifier
    final_results[clf_name] = {
        "f1_micro": f1_micro_test,
        "f1_macro": f1_macro_test,
        "precision_macro": precision_test,
        "recall_macro": recall_test,
        "accuracy": accuracy_test,
        "classification_report": report_test
    }

# Print final test results
print("\nFinal Test Results:")
for clf_name, metrics in final_results.items():
    print(f"\n{clf_name} Metrics:")
    for metric, value in metrics.items():
        if metric != "classification_report":
            print(f"{metric.capitalize()}: {value:.4f}")
    print(f"Classification Report:\n{metrics['classification_report']}")



Final Test Results:

Random Forest Metrics:
F1_micro: 0.7926
F1_macro: 0.6706
Precision_macro: 0.7674
Recall_macro: 0.6502
Accuracy: 0.7926
Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.95      0.87       814
           1       0.73      0.35      0.47       295

    accuracy                           0.79      1109
   macro avg       0.77      0.65      0.67      1109
weighted avg       0.78      0.79      0.76      1109


Bagging Metrics:
F1_micro: 0.7917
F1_macro: 0.6551
Precision_macro: 0.7847
Recall_macro: 0.6366
Accuracy: 0.7917
Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.97      0.87       814
           1       0.78      0.31      0.44       295

    accuracy                           0.79      1109
   macro avg       0.78      0.64      0.66      1109
weighted avg       0.79      0.79      0.76      1109


XGBoost Metrics:
F1_micro: 0.8097
F1_macro: 

In [7]:
# Save results to a pickle file
output_path = "/Users/mac/Desktop/Code_Smell_Detection/data_class/results/codeT5/small/result_codeT5_small_avg.pkl"
with open(output_path, "wb") as f:
    pickle.dump({"cross_val_results": results, "final_results": final_results}, f)
print(f"\nResults saved to {output_path}")



Results saved to /Users/mac/Desktop/Code_Smell_Detection/data_class/results/codeT5/small/result_codeT5_small_avg.pkl
